In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import Xception
from keras.applications.xception import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50 * 3
size = 128
batchsize = 64
lw = 6
STEPS = TOT / EPOCHS / batchsize
NCATS = 340

In [3]:
K.clear_session()

model = Xception(input_shape=(size, size, 1), weights=None, classes=NCATS)
model.compile(optimizer=Adam(lr=0.02), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [4]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw = lw, preprocess_input=preprocess_input)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [5]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 128, 128, 1) (34000, 340)
Validation array memory 2.08 GB


In [6]:
train_datagen = image_generator_xd(size=size, 
                                   batchsize=batchsize, 
                                   lw = lw,
                                   preprocess_input=preprocess_input)

In [ ]:
model_prefix = 'xception{}_lw{}'.format(size, lw)
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
    EarlyStopping(monitor='val_categorical_accuracy', mode='max',patience=20, verbose=1)
]

model.load_weights('models/xception128_lw6128_9.model')

hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/150
5175/5174 [==============================] - 1268s 245ms/step - loss: 1.4905 - categorical_crossentropy: 1.4905 - categorical_accuracy: 0.6555 - top_3_accuracy: 0.8185 - val_loss: 1.5228 - val_categorical_crossentropy: 1.5228 - val_categorical_accuracy: 0.6467 - val_top_3_accuracy: 0.8102

Epoch 00001: val_categorical_accuracy improved from -inf to 0.64674, saving model to ./models/xception128_lw6.model
Epoch 2/150
5175/5174 [==============================] - 1262s 244ms/step - loss: 1.4871 - categorical_crossentropy: 1.4871 - categorical_accuracy: 0.6563 - top_3_accuracy: 0.8191 - val_loss: 1.4600 - val_categorical_crossentropy: 1.4600 - val_categorical_accuracy: 0.6671 - val_top_3_accuracy: 0.8267

Epoch 00002: val_categorical_accuracy improved from 0.64674 to 0.66709, saving model to ./models/xception128_lw6.model
Epoch 3/150
5175/5174 [==============================] - 1260s 244ms/step - loss: 1.4660 - categorical_crossentropy: 1.4660 - categorical_accuracy: 0.6594 - to

5175/5174 [==============================] - 1264s 244ms/step - loss: 1.2267 - categorical_crossentropy: 1.2267 - categorical_accuracy: 0.7164 - top_3_accuracy: 0.8589 - val_loss: 1.1997 - val_categorical_crossentropy: 1.1997 - val_categorical_accuracy: 0.7231 - val_top_3_accuracy: 0.8602

Epoch 00021: val_categorical_accuracy improved from 0.70403 to 0.72306, saving model to ./models/xception128_lw6.model
Epoch 22/150
5175/5174 [==============================] - 1264s 244ms/step - loss: 1.2123 - categorical_crossentropy: 1.2123 - categorical_accuracy: 0.7209 - top_3_accuracy: 0.8609 - val_loss: 1.1857 - val_categorical_crossentropy: 1.1857 - val_categorical_accuracy: 0.7280 - val_top_3_accuracy: 0.8641

Epoch 00022: val_categorical_accuracy improved from 0.72306 to 0.72803, saving model to ./models/xception128_lw6.model
Epoch 23/150
5175/5174 [==============================] - 1260s 244ms/step - loss: 1.2112 - categorical_crossentropy: 1.2112 - categorical_accuracy: 0.7197 - top_3_acc


Epoch 00040: val_categorical_accuracy did not improve from 0.75612
Epoch 41/150
5175/5174 [==============================] - 1265s 245ms/step - loss: 1.0895 - categorical_crossentropy: 1.0895 - categorical_accuracy: 0.7482 - top_3_accuracy: 0.8787 - val_loss: 1.0581 - val_categorical_crossentropy: 1.0581 - val_categorical_accuracy: 0.7558 - val_top_3_accuracy: 0.8829

Epoch 00041: val_categorical_accuracy did not improve from 0.75612
Epoch 42/150
5175/5174 [==============================] - 1265s 244ms/step - loss: 1.0855 - categorical_crossentropy: 1.0855 - categorical_accuracy: 0.7487 - top_3_accuracy: 0.8796 - val_loss: 1.0505 - val_categorical_crossentropy: 1.0505 - val_categorical_accuracy: 0.7579 - val_top_3_accuracy: 0.8844

Epoch 00042: val_categorical_accuracy improved from 0.75612 to 0.75791, saving model to ./models/xception128_lw6.model
Epoch 43/150
5175/5174 [==============================] - 1265s 244ms/step - loss: 1.0907 - categorical_crossentropy: 1.0907 - categorical

3546/5174 [===================>..........] - ETA: 6:23 - loss: 1.0610 - categorical_crossentropy: 1.0610 - categorical_accuracy: 0.7552 - top_3_accuracy: 0.8828

In [ ]:

fold = 9
snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,nb_snapshots=1, 
                                   size = size,
                                   init_lr=1e-3, fold=fold)

callbks = snapshot.get_callbacks(model_prefix = model_prefix)

sgd = SGD(0.002, momentum=0.9, nesterov=True)
sgd = NormalizedOptimizer(sgd, normalization='l2')

model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

In [ ]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

In [ ]:
prefix_swa = ''
try:
    print('using swa weight model')
    model.load_weights('./models/{}_swa_{}.model'.format(model_prefix, fold))
    prefix_swa = 'swa'
except:
    model.load_weights('./models/{}_{}.model'.format(model_prefix, fold))

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw = lw, preprocess_input=preprocess_input)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

In [ ]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

In [ ]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

In [ ]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}{}.csv'.format(model_prefix, prefix_swa),  
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)